# 📘 [Upstage] AI Agent QA 시나리오 데이터셋 구축 

## 콘텐츠 라이선스

저작권: <b><font color='blue'>© 2025 Upstage Co., Ltd. All rights reserved.</font></b>

<b><font color='red'>⚠️ WARNING:</font></b> 본 교육용 코드 및 자료의 모든 지식재산권은 업스테이지에 귀속됩니다.  
본 콘텐츠를 사전 서면 동의 없이 **외부로 유출, 복제, 수정, 배포, 게시하는 행위는 엄격히 금지**됩니다.


## 🎯 프로젝트 개요
본 프로젝트는 **한국 Public API**를 활용하여 도구를 적절히 활용하는 AI Agent 시나리오 데이터셋을 구축합니다.
모델의 Agentic 능력 향상을 위한 고품질 학습 데이터셋을 구축하여, 모델 학습을 위한 데이터 구축 방법론에 대해 학습합니다.  


### 핵심 목표
- **Multi-Turn Tool 호출 시나리오** 구축합니다.
- **오류 복구, 조건부 선택** 등 Agentic Reasoning을 요구하는 내용을 반영합니다. 
- **BFCL 기준 검증** 통과한 고품질 시나리오 데이터셋을 구축하여, 현업에서 실제 평가 벤치마크를 구축하는 방법을 학습합니다. 

---
## 📋 프로젝트 진행 단계

### 1/ 도메인 선정
- 10개 도메인 중 1개 선택 (정부 & 공공기관, 게임 & 엔터테인먼트, 소셜 & 커뮤니케이션, 지도/위치, 네이버, 카카오, 쇼핑, 음식/의료, 날씨/환경, 미디어 & 콘텐츠)

### 2/ Tool 정의
- 최소 2개 이상의 독립적인 Tool 설계
- Real-world 시나리오 활용 가능
- Public API 기반

### 3/ QA 시나리오 데이터 구축
- **조합 1**: Single-turn & Multi-tool-use (최소 2개)
- **조합 2**: Multi-turn & Multi-tool-use (최소 2개)

### 4/ Solar Pro 2 API 호출
- 실제 Solar Pro 2 API로 합성 데이터 생성
- 재시도 로직 포함

### 5/ 정량 평가
- Correct Function Name
- Valid Arguments  
- No Hallucinated Calls

### 6/ 정성 평가 
아래 기준에 따라 정성 평가를 진행합니다.
- 시나리오 적합성 
- 창의성 및 다양성 
- 구체성
- 실행 가능성 
- 윤리·안전성


### 7/ 최종 데이터셋
- 평가 통과한 시나리오만 포함
- JSONL, JSON, CSV 형식으로 저장

---

## 📊 데이터셋 구성

| 조합 | Turn | Tool 사용 | 최소 개수 |
|------|------|-----------|----------|
| 조합 1 | Single-turn | Multi-tool-use | 2개 |
| 조합 2 | Multi-turn | Multi-tool-use | 2개 |

**총 최소**: 4개 시나리오

## 실험 평가 기준

평가 설계 기준 
- 아래 기준 정도로 실험하는 것을 권장 드립니다.

1. 조합 1 ㅣSingle Turn &  Multi Tool Use
   * 정량 50% 
   * 정성 50% 

2. 조합 2 | Multi Turn &  Multi Tool Use
   * 정량 : 30% 
   * 정성 : 70%
     * Multi Turn의 경우, 오류 복구 시나리오를 포함한 데이터의 정성평가의 비중을 더 높여 프로젝트를 진행해보세요! 


---

## 📚 참고 자료
- [활용 데이터 : Public API](https://github.com/yybmion/public-apis-4Kr)
- [BFCL 리더보드](https://github.com/ShishirPatil/gorilla)
- [Nemotron 시나리오 데이터](https://huggingface.co/datasets/nvidia/Nemotron-Post-Training-Dataset-v1)
- [Upstage Console](https://developers.upstage.ai/)

---


# 0️⃣ 환경 설정

이 섹션에서는 프로젝트 실행에 필요한 라이브러리와 BFCL을 설치합니다.

### 필요한 패키지
- `openai`: Solar Pro 2 API 호출
- `requests`: HTTP 요청
- `python-dotenv`: 환경변수 관리
- `tqdm`, `pandas`, `numpy`: 데이터 처리
- BFCL (Berkeley Function Call Leaderboard): 평가 프레임워크

In [56]:
# ============================================================
# requirements.txt 생성
# ============================================================
req = """
python-dotenv>=1.0.1
requests>=2.32.3
tqdm>=4.66.5
rich>=13.8.1
pandas>=2.2.2
numpy>=1.26.4
datasets>=2.21.0
huggingface-hub>=0.24.6
openai==1.52.2
"""
with open("requirements.txt", "w", encoding="utf-8") as f:

    f.write(req.strip() + "\n")

print("✅ requirements.txt 생성 완료")

# 설치
!python -m pip install --upgrade pip
!pip install -r requirements.txt

# Gorilla(BFCL) editable 설치
#!pip install -e external/gorilla
#!pip install gorilla-llm

✅ requirements.txt 생성 완료


In [57]:
# ============================================================
# 필수 라이브러리 임포트
# ============================================================
import os, sys, subprocess, json
import uuid
import time
import random
import csv
import requests

import re
from openai import OpenAI  # Solar Pro 2 API 호출용 (openai==1.52.2)
from pathlib import Path
from datetime import datetime
from dotenv import load_dotenv
from typing import List, Dict, Any, Optional

print("✅ 라이브러리 임포트 완료")



✅ 라이브러리 임포트 완료


In [58]:
# ============================================================
# Solar Pro 2 API Key 설정
# ============================================================
# 
# 📌 API Key 발급 방법:
# 1. https://console.upstage.ai/ 접속
# 2. 회원가입 / 로그인
# 3. Dashboard > API Keys 메뉴에서 새 키 발급
# 4. 아래에 발급받은 키를 입력하세요
#
# ⚠️ 보안 주의사항:
# - API Key를 코드에 직접 입력하는 것은 보안상 위험합니다
# - .env 파일 또는 환경변수 사용을 권장합니다
# - GitHub 등에 업로드 시 키가 노출되지 않도록 주의하세요
# ============================================================

# 방법 1: 직접 입력 (테스트용)
#UPSTAGE_API_KEY = "your_actual_api_key_here"  # ← 여기에 발급받은 키를 입력하세요

# 방법 2: .env 파일 사용 (권장)
# .env 파일을 만들고 다음 내용 추가:
UPSTAGE_API_KEY="your_actual_api_key_here"
env_file = Path(".env")
if env_file.exists():
    load_dotenv()
    UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY", UPSTAGE_API_KEY)
    print("✅ .env 파일에서 API 키를 로드했습니다")
else:
    if UPSTAGE_API_KEY == "YOUR_API_KEY_HERE":
        print("⚠️  API Key를 설정해주세요!")
        print("   위 코드에서 UPSTAGE_API_KEY 값을 실제 키로 변경하세요")
    else:
        print("✅ API Key 설정 완료")

# API 기본 설정
BASE_URL = "https://api.upstage.ai/v1"
print(f"✅ Solar Pro 2 API 엔드포인트: {BASE_URL}")


✅ .env 파일에서 API 키를 로드했습니다
✅ Solar Pro 2 API 엔드포인트: https://api.upstage.ai/v1


In [59]:
# ============================================================
# 출력 디렉토리 생성
# ============================================================
# outputs: 최종 데이터셋 파일 저장 (JSONL, JSON)
# artifacts: 평가 리포트 저장 (CSV)
os.makedirs("outputs", exist_ok=True)
os.makedirs("artifacts", exist_ok=True)

print("✅ 출력 디렉토리 생성 완료")
print("   - outputs/ : 최종 데이터셋 파일")
print("   - artifacts/ : BFCL 평가 리포트")

✅ 출력 디렉토리 생성 완료
   - outputs/ : 최종 데이터셋 파일
   - artifacts/ : BFCL 평가 리포트


## 0-1 Solar Pro 2 호출해보기

https://console.upstage.ai/docs/getting-started

In [60]:
# pip install openai
 
client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1"
)
 
stream = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {
            "role": "user",
            "content": "Hi, how are you?"
        }
    ],
    stream=True,
)
 
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")
 
# Use with stream=False
# print(stream.choices[0].message.content)

Hello! I'm just a large language model, so I don't have feelings, but I'm here and ready to help you. How can I assist you today? 😊  

Is there something specific you'd like to talk about or ask? I'm here to answer questions, provide information, or even have a conversation!  

(And if you're just saying hi, that's totally fine too! How's your day going?)

# 1️⃣ 도메인 선정 & Tool 정의

이 섹션에서는 프로젝트에 사용할 **도메인을 선택**하고, **최소 2개 이상의 Tool**을 정의합니다.

- **도메인 선택**: 10개 도메인 중 1개 선택
- **Tool 개수**: 최소 2개 이상 정의
- **Tool 정의 시 유의사항**: 
  - 독립적이고 중복 없음
  - Real-world 시나리오에 활용 가능
  - Public API 기반
  - sub-/super-set 관계 없음

#### 📋 선택 가능한 도메인 (10개)

| 번호 | 도메인 | 예시 API |
|------|--------|----------|
| 1 | 정부 & 공공기관 | 공공데이터포털, 국방부 |
| 2 | 게임 & 엔터테인먼트 | 게임 API, 스팀 |
| 3 | 소셜 & 커뮤니케이션 | SNS API, 메신저 |
| 4 | 지도/위치 | 카카오맵, 네이버맵 |
| 5 | 네이버 | 블로그, 뉴스, 지역검색 |
| 6 | 카카오 | 카카오톡, 카카오맵 |
| 7 | 쇼핑 | 쇼핑몰 API, 가격비교 |
| 8 | 음식/의료 | 맛집정보, 병원정보 |
| 9 | 날씨/환경 | 기상청, 에어코리아 |
| 10 | 미디어 & 콘텐츠 | 영화, 음악 API |

### 본 베이스라인의 도메인: 정부 & 공공기관 (방산/국방)

이 노트북에서는 **방산/국방 도메인**을 예시로 사용합니다.

#### 정의된 Tool (2개)

| Tool | 설명 | Public API |
|------|------|------------|
| **defense_company_info** | 국내 방산 업체 정보 조회 | Mock Data (실제: 방위사업청) |
| **rok_weapon_info** | 대한민국 무기체계 개요 조회 | Mock Data (실제: 국방부) |


In [62]:
# ============================================================
# 도메인 선택 (여기를 수정하세요!)
# ============================================================
# 10개 도메인 중 1개를 선택합니다.
# 다른 도메인으로 변경하려면 아래 값을 수정하세요.
# ============================================================

SELECTED_DOMAIN = "카카오(검색&맵)"

print(f"✅ 선택한 도메인: {SELECTED_DOMAIN}")

✅ 선택한 도메인: 카카오(검색&맵)


In [88]:
import requests
import json
import os 

# KAKAO_API_KEY = os.getenv("KAKAO_API_KEY", "여기에_발급받은_카카오_REST_API_키를_입력하세요") 
KAKAO_API_KEY = "90f17f034c12feb9e622c75b2204c995"


# --------------------------
# Tool 1: 카카오 블로그 검색 (변경 없음)
# --------------------------
def kakao_search_blog(query: str, sort: str = "accuracy", page: int = 1) -> Dict[str, Any]:
    """
    카카오 검색 API로 '블로그' 문서를 검색합니다. (예: 맛집 후기, 여행 후기)
    """
    url = "https://dapi.kakao.com/v2/search/blog"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": query.encode('utf-8'), "sort": sort, "page": page, "size": 1}
    try:
        response = requests.get(url, headers=headers, params=params, timeout=5)
        response.raise_for_status() 
        return response.json()
    except requests.RequestException as e:
        return {"error": str(e), "query": query}

# --------------------------
# Tool 2: 카카오맵 키워드 장소 검색 (좌표 변환용, 변경 없음)
# --------------------------
def kakao_search_place(query: str) -> Dict[str, Any]:
    """
    카카오맵 API로 '키워드'로 장소를 검색하여 주소, 좌표, 장소명을 반환합니다.
    """
    url = "https://dapi.kakao.com/v2/local/search/keyword.json" 
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": query.encode('utf-8'), "size": 1} 
    try:
        response = requests.get(url, headers=headers, params=params, timeout=5) 
        response.raise_for_status() 
        return response.json()
    except requests.RequestException as e:
        return {"error": str(e), "query": query}
        
# --------------------------
# Tool 3: 카카오맵 경로 탐색 (실제 API 호출로 변경)
# --------------------------
def kakao_map_directions(origin_x: str, origin_y: str, destination_x: str, destination_y: str) -> Dict[str, Any]:
    """
    카카오맵 API로 출발지와 목적지의 '좌표'를 받아 자동차 경로 정보(거리, 소요 시간, 예상 택시비)를 탐색합니다.
    
    Args:
        origin_x (str): 출발지 경도 (X 좌표)
        origin_y (str): 출발지 위도 (Y 좌표)
        destination_x (str): 목적지 경도 (X 좌표)
        destination_y (str): 목적지 위도 (Y 좌표)
        
    Returns:
        Dict: 경로 탐색 실제 결과 (성공 시 routes 리스트, 실패 시 error 포함)
    """
    url = "https://apis-navi.kakaomobility.com/v1/directions" # 카카오내비 API 주소
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    
    # [수정됨] API가 요구하는 'params' 형식으로 변경합니다.
    # 'x,y' 형태의 문자열로 좌표를 조합합니다.
    params = {
        "origin": f"{origin_x},{origin_y}",
        "destination": f"{destination_x},{destination_y}"
    }
    
    try:
        # [수정됨] POST가 아닌 GET 방식으로 요청합니다.
        response = requests.get(url, headers=headers, params=params, timeout=5)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        # 오류 발생 시 어떤 파라미터로 요청했는지 확인하기 쉽도록 params를 error에 포함
        return {"error": str(e), "params": params}

### Tool 형식

- 아래 형식에 따라 작성합니다.

In [14]:
# {
#   "type": "function",
#   "function": {
#     "name": "<함수명>",
#     "description": "<함수의 목적과 사용법>",
#     "parameters": {
#       "type": "object",
#       "properties": {
#         "<argument명>": {
#           "type": "<자료형>",
#           "description": "<argument 설명>"
#         }
#       },
#       "required": ["<필수 argument 리스트>"]
#     }
#   }
# }

In [89]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "kakao_search_blog",
            "description": "카카오 '블로그' 검색 결과를 반환합니다. (예: 맛집 후기, 여행 후기)",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "검색할 블로그 키워드 (예: '제주도 흑돼지 맛집')"},
                    "sort": {"type": "string", "description": "정렬 방식 ('accuracy' 또는 'recency')", "default": "accuracy"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "kakao_search_place",
            "description": "카카오맵 API로 '키워드'(장소명)를 검색하여 정확한 주소와 '좌표(x, y)'를 반환합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "검색할 장소 키워드 (예: '제주국제공항', '흑돼지잡스')"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "kakao_map_directions",
            "description": "카카오맵 API로 출발지와 목적지의 '좌표'를 받아 자동차 경로 정보(거리, 소요 시간, 예상 택시비)를 탐색합니다. 장소 이름으로는 불가능합니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin_x": {"type": "string", "description": "출발지 경도 (X 좌표)"},
                    "origin_y": {"type": "string", "description": "출발지 위도 (Y 좌표)"},
                    "destination_x": {"type": "string", "description": "목적지 경도 (X 좌표)"},
                    "destination_y": {"type": "string", "description": "목적지 위도 (Y 좌표)"}
                },
                "required": ["origin_x", "origin_y", "destination_x", "destination_y"]
            }
        }
    }
]

In [90]:

# ============================================================
# Tool 정의 완료 및 테스트
# ============================================================
print("=" * 60)
print("✅ Tool 정의 완료!")
print("=" * 60)
print(f"선택한 도메인: {SELECTED_DOMAIN}")
print(f"정의된 Tool 개수: {len(tools)}개")
for i, tool in enumerate(tools, 1):
    print(f"  {i}. {tool['function']['name']}")
print("=" * 60)

print("\n[테스트 1] kakao_search_blog(query='흑돼지잡스')")
ex1 = kakao_search_blog(query="흑돼지잡스")
blog_title = ex1.get('documents', [{}])[0].get('title', '제목 없음')
print(f"  -> 블로그 검색 성공, 예시 제목: {blog_title}")

print("\n[테스트 2] kakao_search_place(query='제주국제공항')")
ex2 = kakao_search_place(query="제주국제공항")
origin_doc = ex2.get('documents', [{}])[0]
origin_x, origin_y = origin_doc.get('x'), origin_doc.get('y')
print(f"  -> 출발지 좌표 검색 성공: {origin_doc.get('place_name')} (x: {origin_x}, y: {origin_y})")

print("\n[테스트 3] kakao_search_place(query='흑돼지잡스')")
ex3 = kakao_search_place(query="흑돼지잡스")
dest_doc = ex3.get('documents', [{}])[0]
dest_x, dest_y = dest_doc.get('x'), dest_doc.get('y')
print(f"  -> 목적지 좌표 검색 성공: {dest_doc.get('place_name')} (x: {dest_x}, y: {dest_y})")

print("\n[테스트 4] kakao_map_directions (실제 호출)")
if origin_x and dest_x:
    ex4 = kakao_map_directions(origin_x=origin_x, origin_y=origin_y, destination_x=dest_x, destination_y=dest_y)
    if ex4.get('routes'):
        summary = ex4['routes'][0]['summary']
        print(f"  -> 실제 경로 탐색 성공! 예상 시간: {summary['duration']}초, 예상 택시비: {summary['fare']['taxi']}원")
    else:
        print(f"  -> 경로 탐색 실패: {ex4.get('error', '알 수 없는 오류')}")
else:
    print("  -> 좌표 정보가 없어 경로 탐색을 건너뜁니다.")

print("\n" + "=" * 60)
print("✅ Tool 함수 3개 정상 작동 확인 완료!")
print("=" * 60)

✅ Tool 정의 완료!
선택한 도메인: 카카오(검색&맵)
정의된 Tool 개수: 3개
  1. kakao_search_blog
  2. kakao_search_place
  3. kakao_map_directions

[테스트 1] kakao_search_blog(query='흑돼지잡스')
  -> 블로그 검색 성공, 예시 제목: 제주 구남동 &#39;<b>흑</b><b>돼지</b><b>잡스</b>&#39;, 라마다시티호텔 근처 제주도<b>흑</b><b>돼지</b> 맛집

[테스트 2] kakao_search_place(query='제주국제공항')
  -> 출발지 좌표 검색 성공: 제주국제공항 (x: 126.49272304493574, y: 33.50683984835887)

[테스트 3] kakao_search_place(query='흑돼지잡스')
  -> 목적지 좌표 검색 성공: 흑돼지잡스 (x: 126.53358954829656, y: 33.4894319981105)

[테스트 4] kakao_map_directions (실제 호출)
  -> 실제 경로 탐색 성공! 예상 시간: 810초, 예상 택시비: 9300원

✅ Tool 함수 3개 정상 작동 확인 완료!


# 2️⃣ QA 시나리오 데이터 구축

이 섹션에서는 **Solar Pro 2 API**를 사용하여 시나리오를 자동 생성합니다.

### 📌 생성할 시나리오
1. **Single-turn & Multi-tool-use**: 최소 2개
2. **Multi-turn & Multi-tool-use**: 최소 2개

### 🔄 프로세스
```
사용자 질문 입력
    ↓
Solar Pro 2 API 호출
    ↓
Tool 호출 (자동)
    ↓
Tool 실행 결과 반환
    ↓
최종 답변 생성
```

### ⚠️ 주의사항
- 실제 Solar Pro 2 API를 호출합니다 
- API Key가 올바르게 설정되어 있어야 합니다
- **Assistant 모델의 응답은 `<think>...</think>` 태그가 반드시 포함되어야 합니다**
  - API 호출 시 `include_think=true` 파라미터를 사용하여 태그가 포함된 응답을 받습니다
  - **만약 Assistant 응답에 think 태그가 없으면 후처리 로직 추가가 필요합니다.**
    - 후처리: assistant 응답의 reasoning 부분을 추출하여 `<think>...</think>` 태그로 감싸야 합니다
    - 이는 모델 학습용 데이터 구축의 완성도를 위해 반드시 필요한 요소입니다

## 2-1. Single-turn & Multi-tool-use QA 시나리오 생성

### 📝 Single-turn이란?
- **한 번의 사용자 요청** → **한 번의 AI 답변**
- 중간에 여러 Tool을 호출할 수 있음
- 추가 질문 없이 한 번에 완결

### 🔧 Multi-tool-use란?
- 한 번의 요청을 해결하기 위해 **2개 이상의 Tool을 사용**
- Tool 호출이 순차적 또는 병렬로 실행됨

### 💡 예시
사용자: "한화시스템이 레이더/전자전 도메인을 다루는지 확인하고, 천궁(M-SAM)의 역할도 알려줘."

AI 사고 과정:
1. Tool 1 호출: defense_company_info(query="한화시스템")
2. Tool 2 호출: rok_weapon_info(keyword="천궁")
3. 결과 통합하여 최종 답변 생성
`


In [91]:
# ============================================================
# Solar Pro 2 API를 활용한 합성 데이터셋 구축 : Single Turn   
# ============================================================

BASE_URL = "https://api.upstage.ai/v1"

# ============================================================
# Tool 실행 헬퍼 함수
# ============================================================

def _safe_json(obj) -> str:
    """JSON 직렬화 헬퍼"""
    try:
        return json.dumps(obj, ensure_ascii=False)
    except Exception:
        return json.dumps({"_error": "non-serializable"}, ensure_ascii=False)

def call_solar_api(messages: List[Dict[str, Any]], 
                   tools_spec: list,
                   temperature: float = 0.1, 
                   max_tokens: int = 800,
                   max_retries: int = 3) -> Dict[str, Any]:
    """
    Solar Pro 2 Chat Completions API 호출 (재시도 로직 포함)
    """
    url = f"{BASE_URL}/solar/chat/completions?include_think=true" # <think>...</think> 태그 추가 
    headers = {
        "Authorization": f"Bearer {UPSTAGE_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "solar-pro2",
        "messages": messages,
        "tools": tools_spec,
        "tool_choice": "auto",
        "temperature": temperature,
        "reasoning_effort" : "high", 
        "max_tokens": max_tokens,
    }
    
    for attempt in range(1, max_retries + 1):
        try:
            r = requests.post(url, headers=headers, json=payload, timeout=60)
            if r.status_code in (429,) or r.status_code >= 500:
                raise requests.HTTPError(f"{r.status_code} {r.text[:300]}")
            r.raise_for_status()
            return r.json()
        except requests.HTTPError as e:
            if attempt < max_retries:
                wait_time = (2 ** (attempt - 1)) + random.random()
                print(f"   ⚠️  재시도 {attempt}/{max_retries} (대기: {wait_time:.1f}초)")
                time.sleep(wait_time)
            else:
                print(f"   ❌ API 호출 실패: {e}")
                raise


def execute_tool(name: str, args: dict) -> str:
    """
    실제 Tool 실행 (카카오 검색&맵)
    """
    if name == "kakao_search_blog":
        res = kakao_search_blog(**args) 
    elif name == "kakao_search_place":
        res = kakao_search_place(**args)
    elif name == "kakao_map_directions":
        res = kakao_map_directions(**args)
    else:
        res = {"error": f"unknown tool: {name}"}
    
    out = _safe_json(res)
    # 길이 제한 (API 오류 방지)
    if len(out) > 4000:
        out = out[:4000] + f' ... (truncated {len(out)-4000} chars)'
    return out


print("✅ Solar Pro 2 API 설정 완료") 



# ============================================================
# Single-turn 시나리오 생성 함수
# ============================================================

def generate_single_turn_scenario(user_query: str,
                                   task_description: str,
                                   tools_spec: list,
                                   row_idx: Optional[int] = None) -> Dict[str, Any]:
    """
    Single-turn & Multi-tool-use 시나리오 생성.

    prompt 그리고 시나리오 생성 로직 등을 변경해보면서 자신만의 시나리오를 생성해보세요! 
    """
    system_prompt = (
        """
        너는 어떤 도구를 사용해야할 지 판단하고, 필요한 도구를 호출하는 도구 사용 전략가야.
        도구 인자는 스키마에 맞춰라. 여러 도구를 사용해야 할 경우 순차적으로 호출하라.
        항상 너의 사고 과정을 <think>...</think> 태그 안에 명시적으로 작성한 후, 최종 답변을 작성해야해.
        절대 내부 reasoning을 숨기지 말고 반드시 <think>...</think> 태그를 포함해 출력하세요.
        예시:
        <think>
        사용자의 요청을 단계별로 처리해보겠습니다. 
        "사용자 요청 분석: [사용자가 무엇을 원하는지 자세히 분석]\n"
        "필요한 도구 판단: [어떤 도구가 필요한지와 각 도구를 선택한 이유]\n"
        "각 도구 파라미터 설정: [각 도구의 파라미터를 어떻게 설정할지 상세히]\n"
        "도구 호출 순서 및 이유: [여러 도구를 호출한다면 순서와 각 순서의 이유]\n"
        </think>
        """
    )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    
    # messages 배열 (샘플 형식에 맞춤)
    messages_list = [
        {
            "role": "user",
            "content": user_query,
            "tool_calls": []
        }
    ]
    
    # 최대 5번 반복 (도구 호출)
    for step in range(5):
        try:
            data = call_solar_api(messages, tools_spec)
            msg = data["choices"][0]["message"]
            
            # Tool calls 추출
            tool_calls = []
            if isinstance(msg.get("tool_calls"), list):
                for tc in msg["tool_calls"]:
                    fn = tc.get("function") or {}
                    name = fn.get("name") or tc.get("name")
                    args_raw = fn.get("arguments") or tc.get("arguments") or "{}"
                    try:
                        args = json.loads(args_raw) if isinstance(args_raw, str) else args_raw
                    except Exception:
                        args = {}
                    tool_calls.append({"id": tc.get("id"), "name": name, "arguments": args})
            
            content = msg.get("content") or ""
            
            # Assistant 메시지 형식 구성
            assistant_tool_calls = []
            if tool_calls:
                assistant_tool_calls = [
                    {
                        "id": tc["id"],
                        "type": "function",
                        "function": {
                            "name": tc["name"],
                            "arguments": json.dumps(tc["arguments"], ensure_ascii=False)
                        }
                    } for tc in tool_calls
                ]
            
            # Assistant 메시지 추가
            messages_list.append({
                "role": "assistant",
                "content": content,
                "tool_calls": assistant_tool_calls
            })
            messages.append(msg)
            
            # 도구 호출이 없으면 최종 답변으로 종료
            if not tool_calls:
                break
            
            # 각 도구 실행 및 tool 메시지 추가
            for tc in tool_calls:
                result = execute_tool(tc["name"], tc.get("arguments", {}))
                
                tool_message = {
                    "role": "tool",
                    "tool_call_id": tc["id"],
                    "content": result
                }
                messages.append(tool_message)
                
                # 샘플 형식에 맞춰 tool 메시지 추가 (tool_calls 필드 포함)
                messages_list.append({
                    "role": "tool",
                    "content": result,
                    "tool_calls": []
                })
            
            time.sleep(0.5)
            
        except Exception as e:
            print(f"   ❌ 오류 발생: {e}")
            break
    
    # metadata를 JSON 문자열로 변환
    metadata_dict = {"tools": tools_spec}
    metadata_str = json.dumps(metadata_dict, ensure_ascii=False)
    
    # 샘플 형식에 맞춰 반환
    result = {
        "row": {
            "messages": messages_list,
            "metadata": metadata_str
        }
    }
    
    # row_idx가 제공된 경우에만 추가
    if row_idx is not None:
        result["row_idx"] = row_idx
    
    return result


def save_jsonl(items: List[Dict[str, Any]], path: str):
    os.makedirs(os.path.dirname(path) or ".", exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        for it in items:
            f.write(json.dumps(it, ensure_ascii=False) + "\n")
    print(f"✅ saved: {path}")

✅ Solar Pro 2 API 설정 완료


In [86]:
# ============================================================
# Single-turn 시나리오 생성 실행
# ============================================================
print("\n[1/2] Single-turn & Multi-tool-use 시나리오 생성 중...")
single_scenarios = []

# 시나리오 1
print("   [1-1] 시나리오 1 생성...")
scenario1 = generate_single_turn_scenario(
    user_query="제주도 여행 계획 중이야. 카카오 블로그 검색으로 '제주도 흑돼지 맛집'을 찾아서 식당 이름과 주소를 알려줘. 그리고 카카오내비로 '제주국제공항'에서 그 식당까지 가는 자동차 예상 소요 시간과 택시비를 계산해 줘.",
    task_description="카카오 블로그 검색, 카카오맵 장소 검색(좌표변환), 카카오맵 경로탐색(택시비) 3개 Tool 순차 호출",
    tools_spec=tools,
    row_idx=1
)
single_scenarios.append(scenario1)
print(f"   ✅ 생성 완료 (row_idx: {scenario1.get('row_idx', 'N/A')})")

# 시나리오 2
print("   [1-2] 시나리오 2 생성...")
scenario2 = generate_single_turn_scenario(
    user_query="서울에서 '소금빵'으로 제일 유명한 빵집을 '카카오 블로그'에서 찾아줘. 그리고 '강남역'에서 그 빵집까지 택시로 가는 데 예상 소요 시간이랑 요금이 얼마인지 '카카오맵'으로 계산해 줘.",
    task_description="블로그 검색(소금빵), 장소 좌표 변환(2회), 경로 탐색(택시비) 3개 Tool 순차 호출",
    tools_spec=tools,
    row_idx=2
)
single_scenarios.append(scenario2)
print(f"   ✅ 생성 완료 (row_idx: {scenario2.get('row_idx', 'N/A')})")

# 저장
save_jsonl(single_scenarios, "data/kakao_single_turn_solar.jsonl")
print(f"✅ 'kakao_single_turn_solar.jsonl' 파일로 저장 완료.")


[1/2] Single-turn & Multi-tool-use 시나리오 생성 중...
   [1-1] 시나리오 1 생성...
   ✅ 생성 완료 (row_idx: 1)
   [1-2] 시나리오 2 생성...
   ✅ 생성 완료 (row_idx: 2)
✅ saved: data/kakao_single_turn_solar.jsonl
✅ 'kakao_single_turn_solar.jsonl' 파일로 저장 완료.


## 2-2. Multi-turn & Multi-tool-use 시나리오 생성

### 📝 Multi-turn이란?
- **여러 번의 대화 주고받기**
- 사용자 요청 → AI 답변 → 사용자 추가 질문 → AI 답변 (반복)
- 이전 맥락을 유지하면서 대화 진행

### 🔧 Multi-tool-use + Multi-turn
- 여러 턴에 걸쳐 다양한 Tool을 사용
- **오류 복구 시나리오** 포함 (의도적으로 실패 → 재시도)
- **조건부 선택** 가능

### 💡 예시
```
턴 1)
사용자: "한화시스템이 레이더/전자전 도메인을 하는지 먼저 확인해줘."
AI: defense_company_info() 호출 → 답변

턴 2)
사용자: "그럼 중거리 방공체계 천궁의 개요도 알려줘."
AI: rok_weapon_info() 호출 (실패) → 재시도 → 성공 → 답변

턴 3)
사용자: "천궁을 만든 회사 정보도 같이 보여줘."
AI: defense_company_info() 호출 → 답변
```

In [92]:
# ============================================================
# Multi-turn 시나리오 생성 함수
# ============================================================

def generate_multi_turn_scenario(initial_query: str,
                                  follow_up_queries: List[str],
                                  task_description: str,
                                  tools_spec: list,
                                  row_idx: Optional[int] = None,
                                  max_steps: int = 4) -> Dict[str, Any]:
    """
    Multi-turn & Multi-tool-use 시나리오 생성.
    prompt 그리고 시나리오 생성 로직 등을 변경해보면서 자신만의 시나리오를 생성해보세요! 

    """
    system_prompt = (
        """
        너는 한국어로 대화한다. 여러 턴에 걸쳐 도구를 호출한다.
        인자가 틀리면 원인을 설명하고 인자를 수정해 재시도하라.
        너는 어떤 도구를 사용해야할 지 판단하고, 필요한 도구를 호출하는 도구 사용 전략가야.
        도구 인자는 스키마에 맞춰라. 여러 도구를 사용해야 할 경우 순차적으로 호출하라.
        항상 너의 사고 과정을 <think>...</think> 태그 안에 명시적으로 작성한 후, 최종 답변을 작성해야해.
        절대 내부 reasoning을 숨기지 말고 반드시 <think>...</think> 태그를 포함해 출력하세요.
        예시:
        <think>
        사용자의 요청을 단계별로 처리해보겠습니다. 
        "사용자 요청 분석: [사용자가 무엇을 원하는지 자세히 분석]\n"
        "필요한 도구 판단: [어떤 도구가 필요한지와 각 도구를 선택한 이유]\n"
        "각 도구 파라미터 설정: [각 도구의 파라미터를 어떻게 설정할지 상세히]\n"
        "도구 호출 순서 및 이유: [여러 도구를 호출한다면 순서와 각 순서의 이유]\n"
        </think>
        """ 
    )
    
    messages = [{"role": "system", "content": system_prompt}]
    # messages 배열 (샘플 형식에 맞춤)
    messages_list = []
    
    all_queries = [initial_query] + follow_up_queries
    
    for turn_idx, query in enumerate(all_queries):
        # User 메시지 추가
        user_message = {
            "role": "user",
            "content": query,
            "tool_calls": []
        }
        messages_list.append(user_message)
        messages.append({"role": "user", "content": query})
        
        # 각 턴당 최대 3번 시도 (오류 복구)
        for attempt in range(3):
            try:
                data = call_solar_api(messages, tools_spec)
                msg = data["choices"][0]["message"]
                
                # Tool calls 추출
                tool_calls = []
                if isinstance(msg.get("tool_calls"), list):
                    for tc in msg["tool_calls"]:
                        fn = tc.get("function") or {}
                        name = fn.get("name") or tc.get("name")
                        args_raw = fn.get("arguments") or tc.get("arguments") or "{}"
                        try:
                            args = json.loads(args_raw) if isinstance(args_raw, str) else args_raw
                        except Exception:
                            args = {}
                        tool_calls.append({"id": tc.get("id"), "name": name, "arguments": args})
                
                content = msg.get("content") or ""
                
                # Assistant 메시지 형식 구성
                assistant_tool_calls = []
                if tool_calls:
                    assistant_tool_calls = [
                        {
                            "id": tc["id"],
                            "type": "function",
                            "function": {
                                "name": tc["name"],
                                "arguments": json.dumps(tc["arguments"], ensure_ascii=False)
                            }
                        } for tc in tool_calls
                    ]
                
                # Assistant 메시지 추가
                messages_list.append({
                    "role": "assistant",
                    "content": content,
                    "tool_calls": assistant_tool_calls
                })
                messages.append(msg)
                
                # 도구 호출이 없으면 최종 답변으로 종료
                if not tool_calls:
                    break
                
                # 각 도구 실행 및 tool 메시지 추가
                has_error = False
                for tc in tool_calls:
                    # 오류 복구 시나리오: 두 번째 턴의 첫 시도에서 의도적 실패
                    if turn_idx == 1 and attempt == 0:
                        result = json.dumps({
                            "error": "Invalid tool call",
                            "details": {
                                "missing": [],
                                "invalid": {
                                    "parameter": "Expected integer, got string"
                                }
                            }
                        }, ensure_ascii=False)
                        has_error = True
                    else:
                        result = execute_tool(tc["name"], tc.get("arguments", {}))
                    
                    tool_message = {
                        "role": "tool",
                        "tool_call_id": tc["id"],
                        "content": result
                    }
                    messages.append(tool_message)
                    
                    # 샘플 형식에 맞춰 tool 메시지 추가 (tool_calls 필드 포함)
                    messages_list.append({
                        "role": "tool",
                        "content": result,
                        "tool_calls": []
                    })
                
                # 에러가 있으면 재시도 (다음 API 호출에서 assistant가 에러를 분석하고 재시도)
                if has_error:
                    continue
                else:
                    break
                    
            except Exception as e:
                print(f"   ❌ 턴 {turn_idx+1} 오류: {e}")
                break
            
            time.sleep(0.5)
    
    # metadata를 JSON 문자열로 변환
    metadata_dict = {"tools": tools_spec}
    metadata_str = json.dumps(metadata_dict, ensure_ascii=False)
    
    # 샘플 형식에 맞춰 반환
    result = {
        "row": {
            "messages": messages_list,
            "metadata": metadata_str
        }
    }
    
    # row_idx가 제공된 경우에만 추가
    if row_idx is not None:
        result["row_idx"] = row_idx
    
    return result


In [97]:
# ============================================================
# Multi-turn 시나리오 생성 실행 (카카오 도메인, 3턴으로 수정)
# ============================================================
print("\n[2/2] Multi-turn & Multi-tool-use 시나리오 생성 중...")
multi_scenarios = []

# 시나리오 1: 제주도 흑돼지 (3턴으로 수정)
print("   [2-1] 시나리오 1 생성...")
scenario1 = generate_multi_turn_scenario(
    initial_query="카카오 블로그에서 '제주도 흑돼지 맛집' 후기를 검색해서 ,그 '식당 이름'을 알려줘.", # 1턴: Tool 1 (블로그)
    follow_up_queries=[
        "좋아. 그 식당의 정확한 좌표(x, y)가 궁금해. '카카오맵'에서 검색해 줘.",
        "이번에는 '제주국제공항'의 좌표(x, y)를 찾아줘",
        "그럼 '제주국제공항'에서 방금 찾은 그 식당까지 택시로 가는 예상 소요 시간이랑 요금도 알려줘." # 3턴: Tool 2, Tool 3 (장소 검색 + 경로 탐색)
    ],
    task_description="[조합 2] 1턴:블로그 검색 -> 2턴:좌표 검색 -> 3턴:좌표 검색- > 4턴:경로 탐색",
    tools_spec=tools,
    row_idx=1 
)
multi_scenarios.append(scenario1)
num_turns = len([m for m in scenario1['row']['messages'] if m.get('role') == 'user'])
print(f"   ✅ 생성 완료 (row_idx: {scenario1.get('row_idx', 'N/A')}, 턴: {num_turns})")

# 시나리오 2: 서울 소금빵 (3턴으로 수정)
print("   [2-2] 시나리오 2 생성...")
scenario2 = generate_multi_turn_scenario(
    initial_query="강남에서 '소금빵'으로 제일 유명한 빵집을 '카카오 블로그'에서 검색해서, 빵집 이름을 알려줘.", # 1턴: Tool 1 (블로그)
    follow_up_queries=[
        "그 빵집 '강남역' 근처에 있는 거 맞아? '카카오맵'에서 '강남역 11번 출구' 주소랑 좌표 좀 찾아줘.", # 2턴: Tool 2 (장소 검색)
        "아하. 그럼 '강남역 11번 출구'에서 방금 찾은 그 빵집까지 택시로 가는 예상 시간이랑 요금은 얼마야?" # 3턴: Tool 2, Tool 3 (장소 검색 + 경로 탐색)
    ],
    task_description="[조합 2] 1턴:블로그 검색 -> 2턴:좌표 검색 -> 3턴:좌표 검색+경로 탐색",
    tools_spec=tools,
    row_idx=2
)
multi_scenarios.append(scenario2)
num_turns = len([m for m in scenario2['row']['messages'] if m.get('role') == 'user'])
print(f"   ✅ 생성 완료 (row_idx: {scenario2.get('row_idx', 'N/A')}, 턴: {num_turns})")

# 저장
save_jsonl(multi_scenarios, "data/kakao_multi_turn_solar.jsonl") 
print(f"✅ 'kakao_multi_turn_solar.jsonl' 파일로 저장 완료.")


[2/2] Multi-turn & Multi-tool-use 시나리오 생성 중...
   [2-1] 시나리오 1 생성...
   ✅ 생성 완료 (row_idx: 1, 턴: 4)
   [2-2] 시나리오 2 생성...
   ✅ 생성 완료 (row_idx: 2, 턴: 3)
✅ saved: data/kakao_multi_turn_solar.jsonl
✅ 'kakao_multi_turn_solar.jsonl' 파일로 저장 완료.


# 3️⃣ BFCL 평가 (Berkeley Function Call Leaderboard)

이 섹션에서는 생성된 시나리오를 **BFCL 기준**으로 평가합니다.

### 📊 평가 기준 

- Single Turn

| 평가 항목 | 설명 | 방법 |
|----------|------|------|
| **Correct Function Name** | 올바른 함수 호출 여부 | Rule-based |
| **Valid Arguments** | 인자 타입/포맷 정확성 | Rule-based |
| **No Hallucinated Calls** | 정의되지 않은 함수 방지 | Rule-based |


### ✅ 통과 조건
- 각 평가 기준을 **모두** 통과해야 최종 데이터셋에 포함됩니다

### 📁 결과물
- **CSV**: 전체 평가 리포트 (`artifacts/bfcl_defense_report_multi.csv`)
- **JSONL**: 통과한 시나리오만 (`artifacts/bfcl_defense_passes_multi.jsonl`)

### 3-1 Single Turn 평가

evaluate.py를 사용하여 Single Turn 시나리오를 평가합니다.

In [ ]:
# ============================================================
# evaluate.py 모듈 import
# ============================================================
import sys
from pathlib import Path

# eval 디렉토리를 path에 추가
eval_dir = Path("eval")
if str(eval_dir.absolute()) not in sys.path:
    sys.path.insert(0, str(eval_dir.absolute()))

# evaluate 모듈 import
from evaluate import evaluate_entry, load_data

print("✅ evaluate.py 모듈 로드 완료")


✅ evaluate.py 모듈 로드 완료


In [ ]:
# ============================================================
# Single Turn 시나리오 평가
# ============================================================
print("\n" + "=" * 60)
print("📊 Single Turn 시나리오 평가")
print("=" * 60)

# Single turn 데이터 로드
single_turn_data = load_data("data/defense_single_turn_solar.jsonl")

single_turn_results = []
for entry in single_turn_data:
    entry_id = entry.get("row_idx") or entry.get("id", "unknown")
    print(f"\n[평가 중] ID: {entry_id}")
    
    try:
        result = evaluate_entry(entry)
        result["id"] = entry_id
        single_turn_results.append({
            "id": entry_id,
            **result
        })
        
        # 결과 출력
        cfn = '✅' if result['correct_function_name'] else '❌'
        va = '✅' if result['valid_arguments'] else '❌'
        nhc = '✅' if result['no_hallucinated_calls'] else '❌'
        pass_status = '✅ PASS' if result['pass'] else '❌ FAIL'
        
        print(f"   - Correct Function Name: {cfn}")
        print(f"   - Valid Arguments: {va}")
        print(f"   - No Hallucinated Calls: {nhc}")
        print(f"   - 최종 결과: {pass_status}")
        print(f"   - 턴 수: {result.get('num_turns', 0)}, 총 도구 호출: {result.get('total_tool_calls', 0)}")
        
        if result.get('errors'):
            print(f"   - 오류: {', '.join(result['errors'][:2])}")  # 처음 2개만 표시
        
    except Exception as e:
        print(f"   ❌ 평가 중 오류 발생: {e}")
        single_turn_results.append({
            "id": entry_id,
            "correct_function_name": False,
            "valid_arguments": False,
            "no_hallucinated_calls": False,
            "pass": False,
            "errors": [f"평가 중 오류 발생: {str(e)}"]
        })

# 요약 출력
total_single = len(single_turn_results)
passed_single = sum(1 for r in single_turn_results if r.get("pass"))

print("\n" + "=" * 60)
print("📊 Single Turn 평가 결과 요약")
print("=" * 60)
print(f"총 평가 항목: {total_single}")
print(f"✅ 통과: {passed_single} ({passed_single/total_single*100:.1f}%)" if total_single > 0 else "✅ 통과: 0")
print(f"❌ 실패: {total_single - passed_single} ({(total_single-passed_single)/total_single*100:.1f}%)" if total_single > 0 else "❌ 실패: 0")

if total_single > 0:
    print(f"\n📋 평가 항목별 통과율:")
    print(f"  - Correct Function Name: {sum(1 for r in single_turn_results if r.get('correct_function_name'))}/{total_single}")
    print(f"  - Valid Arguments: {sum(1 for r in single_turn_results if r.get('valid_arguments'))}/{total_single}")
    print(f"  - No Hallucinated Calls: {sum(1 for r in single_turn_results if r.get('no_hallucinated_calls'))}/{total_single}")
print("=" * 60)



📊 Single Turn 시나리오 평가

[평가 중] ID: 1
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 1, 총 도구 호출: 2

[평가 중] ID: 2
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 1, 총 도구 호출: 2

📊 Single Turn 평가 결과 요약
총 평가 항목: 2
✅ 통과: 2 (100.0%)
❌ 실패: 0 (0.0%)

📋 평가 항목별 통과율:
  - Correct Function Name: 2/2
  - Valid Arguments: 2/2
  - No Hallucinated Calls: 2/2


### 3-2 Multi Turn 평가

evaluate.py를 사용하여 Multi Turn 시나리오를 평가합니다.

In [ ]:
# ============================================================
# Multi Turn 시나리오 평가
# ============================================================
print("\n" + "=" * 60)
print("📊 Multi Turn 시나리오 평가")
print("=" * 60)

# Multi turn 데이터 로드
multi_turn_data = load_data("data/defense_multi_turn_solar.jsonl")

multi_turn_results = []
for entry in multi_turn_data:
    entry_id = entry.get("row_idx") or entry.get("id", "unknown")
    print(f"\n[평가 중] ID: {entry_id}")
    
    try:
        result = evaluate_entry(entry)
        result["id"] = entry_id
        multi_turn_results.append({
            "id": entry_id,
            **result
        })
        
        # 결과 출력
        cfn = '✅' if result['correct_function_name'] else '❌'
        va = '✅' if result['valid_arguments'] else '❌'
        nhc = '✅' if result['no_hallucinated_calls'] else '❌'
        pass_status = '✅ PASS' if result['pass'] else '❌ FAIL'
        
        print(f"   - Correct Function Name: {cfn}")
        print(f"   - Valid Arguments: {va}")
        print(f"   - No Hallucinated Calls: {nhc}")
        print(f"   - 최종 결과: {pass_status}")
        print(f"   - 턴 수: {result.get('num_turns', 0)}, 총 도구 호출: {result.get('total_tool_calls', 0)}")
        
        # 턴별 결과 출력
        if result.get('turn_results'):
            print(f"   - 턴별 결과:")
            for turn_result in result['turn_results']:
                turn_num = turn_result.get('turn', 0)
                turn_pass = '✅' if turn_result.get('pass') else '❌'
                print(f"     * 턴 {turn_num}: {turn_pass} (도구 호출: {turn_result.get('num_tool_calls', 0)})")
        
        if result.get('errors'):
            print(f"   - 오류: {', '.join(result['errors'][:2])}")  # 처음 2개만 표시
        
    except Exception as e:
        print(f"   ❌ 평가 중 오류 발생: {e}")
        multi_turn_results.append({
            "id": entry_id,
            "correct_function_name": False,
            "valid_arguments": False,
            "no_hallucinated_calls": False,
            "pass": False,
            "errors": [f"평가 중 오류 발생: {str(e)}"]
        })

# 요약 출력
total_multi = len(multi_turn_results)
passed_multi = sum(1 for r in multi_turn_results if r.get("pass"))

print("\n" + "=" * 60)
print("📊 Multi Turn 평가 결과 요약")
print("=" * 60)
print(f"총 평가 항목: {total_multi}")
print(f"✅ 통과: {passed_multi} ({passed_multi/total_multi*100:.1f}%)" if total_multi > 0 else "✅ 통과: 0")
print(f"❌ 실패: {total_multi - passed_multi} ({(total_multi-passed_multi)/total_multi*100:.1f}%)" if total_multi > 0 else "❌ 실패: 0")

if total_multi > 0:
    print(f"\n📋 평가 항목별 통과율:")
    print(f"  - Correct Function Name: {sum(1 for r in multi_turn_results if r.get('correct_function_name'))}/{total_multi}")
    print(f"  - Valid Arguments: {sum(1 for r in multi_turn_results if r.get('valid_arguments'))}/{total_multi}")
    print(f"  - No Hallucinated Calls: {sum(1 for r in multi_turn_results if r.get('no_hallucinated_calls'))}/{total_multi}")
print("=" * 60)



📊 Multi Turn 시나리오 평가

[평가 중] ID: 3
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 3, 총 도구 호출: 3
   - 턴별 결과:
     * 턴 1: ✅ (도구 호출: 1)
     * 턴 2: ✅ (도구 호출: 1)
     * 턴 3: ✅ (도구 호출: 1)

[평가 중] ID: 4
   - Correct Function Name: ✅
   - Valid Arguments: ✅
   - No Hallucinated Calls: ✅
   - 최종 결과: ✅ PASS
   - 턴 수: 4, 총 도구 호출: 4
   - 턴별 결과:
     * 턴 1: ✅ (도구 호출: 1)
     * 턴 2: ✅ (도구 호출: 1)
     * 턴 3: ✅ (도구 호출: 1)
     * 턴 4: ✅ (도구 호출: 1)

📊 Multi Turn 평가 결과 요약
총 평가 항목: 2
✅ 통과: 2 (100.0%)
❌ 실패: 0 (0.0%)

📋 평가 항목별 통과율:
  - Correct Function Name: 2/2
  - Valid Arguments: 2/2
  - No Hallucinated Calls: 2/2


# 4️⃣ 최종 데이터셋 평가 & 저장

이 섹션에서는 생성된 **Single-turn과 Multi-turn 시나리오를 모두 평가**하고 데이터셋을 저장합니다.
해당 데이터셋을 바탕으로 정성평가를 진행합니다

### 📊 정성 평가 기준
- 정량 평가 기준 통과한 데이터셋을 바탕으로 정성 평가를 진행합니다.

| 평가 항목         | 배점 | 평가 기준                                                                                                       |
| ------------- | -- | ----------------------------------------------------------------------------------------------------------- |
| **시나리오 적합성**  | 15 | - 도메인/Task와 일관성 있는가?<br>- 실제 서비스/공공 API 활용 맥락을 반영했는가?                                                       |
| **창의성 및 다양성** | 15 | - 단순 반복적인 QA가 아닌 새로운 활용 아이디어를 제시했는가?<br>- 시나리오가 다양한 상황·조건(오류, 조건 분기, 동적 검색 등)을 폭넓게 커버했는가?                   |
| **구체성**       | 15 | - 상황·조건(시간/위치/제약 등)이 구체적으로 기술되었는가?<br>- 실제 API 호출로 매핑 가능성이 명확한가?                                            |
| **실행 가능성**    | 25 | - LLM + 한국 API 기반으로 실제 실행 가능한 수준인가?<br>- 너무 추상적이거나 불가능한 요청은 아닌가?<br>- 실패(에러, 잘못된 호출) 이후 성공적으로 복구했는가? *(중요)* |
| **윤리·안전성**    | 10 | - 개인정보, 정치/사회적 민감 이슈를 피했는가?<br>- AI 윤리적 문제(차별, 편향 등) 우려가 없는가?                                               |
                                |






### 📁 최종 결과물
- `outputs/final_dataset.jsonl`: 정량평가 통과한 시나리오 
- `outputs/final_dataset.json`: 데이터셋 + 통계
- `artifacts/bfcl_evaluation_report.csv`: 정량 평가 리포트


In [ ]:
# ============================================================
# 최종 데이터셋 평가 & 저장
# ============================================================

print("\n" + "=" * 60)
print("📊 전체 평가 결과 통합")
print("=" * 60)

# 모든 평가 결과 통합
all_results = single_turn_results + multi_turn_results
all_evaluation_results = []

# 통과한 시나리오만 원본 데이터와 매칭
passed_scenarios = []

# Single turn 통과 항목
for result in single_turn_results:
    if result.get("pass"):
        # 원본 데이터 찾기
        entry_id = result.get("id")
        original_entry = next((e for e in single_turn_data if (e.get("row_idx") or e.get("id")) == entry_id), None)
        if original_entry:
            original_entry["evaluation"] = result
            passed_scenarios.append(original_entry)

# Multi turn 통과 항목
for result in multi_turn_results:
    if result.get("pass"):
        # 원본 데이터 찾기
        entry_id = result.get("id")
        original_entry = next((e for e in multi_turn_data if (e.get("row_idx") or e.get("id")) == entry_id), None)
        if original_entry:
            original_entry["evaluation"] = result
            passed_scenarios.append(original_entry)

# 평가 결과 포맷팅 (CSV용)
for result in all_results:
    all_evaluation_results.append({
        "id": result.get("id"),
        "correct_function_name": result.get("correct_function_name", False),
        "valid_arguments": result.get("valid_arguments", False),
        "no_hallucinated_calls": result.get("no_hallucinated_calls", False),
        "pass": result.get("pass", False),
        "num_turns": result.get("num_turns", 0),
        "total_tool_calls": result.get("total_tool_calls", 0)
    })

# 통계 계산
total = len(all_results)
passed = sum(1 for r in all_results if r.get("pass"))
failed = total - passed

single_turn_passed = sum(1 for r in single_turn_results if r.get("pass"))
multi_turn_passed = sum(1 for r in multi_turn_results if r.get("pass"))

print(f"전체 시나리오: {total}개")
print(f"  - Single Turn: {len(single_turn_results)}개 (통과: {single_turn_passed}개)")
print(f"  - Multi Turn: {len(multi_turn_results)}개 (통과: {multi_turn_passed}개)")
print(f"\n✅ 총 통과: {passed}개 ({passed/total*100:.1f}%)" if total > 0 else "\n✅ 총 통과: 0개")
print(f"❌ 총 실패: {failed}개 ({failed/total*100:.1f}%)" if total > 0 else "❌ 총 실패: 0개")
print("=" * 60)

# --------------------------
# 최종 데이터셋 저장
# --------------------------
print("\n" + "=" * 60)
print("💾 최종 데이터셋 저장")
print("=" * 60)

# 1. JSONL 형식으로 통과 항목만 저장
output_jsonl = "outputs/final_dataset.jsonl"
with open(output_jsonl, "w", encoding="utf-8") as f:
    for scenario in passed_scenarios:
        f.write(json.dumps(scenario, ensure_ascii=False) + "\n")
print(f"✅ JSONL 저장: {output_jsonl} (통과 항목: {len(passed_scenarios)}개)")

# 2. JSON 형식으로 전체 저장
final_dataset = {
    "project": "Upstage AI Agent QA 시나리오 데이터셋 구축",
    "domain": SELECTED_DOMAIN,
    "created_at": datetime.now().isoformat(),
    "tools": tools,
    "scenarios": passed_scenarios,
    "statistics": {
        "total_generated": total,
        "passed_evaluation": len(passed_scenarios),
        "single_turn_total": len(single_turn_results),
        "single_turn_passed": single_turn_passed,
        "multi_turn_total": len(multi_turn_results),
        "multi_turn_passed": multi_turn_passed,
        "pass_rate": f"{passed/total*100:.1f}%" if total > 0 else "0%"
    }
}

output_json = "outputs/final_dataset.json"
with open(output_json, "w", encoding="utf-8") as f:
    json.dump(final_dataset, f, ensure_ascii=False, indent=2)
print(f"✅ JSON 저장: {output_json}")

# 3. 평가 리포트 CSV 저장
report_csv = "artifacts/bfcl_evaluation_report.csv"
with open(report_csv, "w", newline="", encoding="utf-8") as f:
    fieldnames = [
        "id", "correct_function_name", "valid_arguments", 
        "no_hallucinated_calls", "pass", "num_turns", "total_tool_calls"
    ]
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for result in all_evaluation_results:
        row = {
            "id": result.get("id"),
            "correct_function_name": 1 if result.get("correct_function_name") else 0,
            "valid_arguments": 1 if result.get("valid_arguments") else 0,
            "no_hallucinated_calls": 1 if result.get("no_hallucinated_calls") else 0,
            "pass": 1 if result.get("pass") else 0,
            "num_turns": result.get("num_turns", 0),
            "total_tool_calls": result.get("total_tool_calls", 0)
        }
        writer.writerow(row)
print(f"✅ CSV 리포트: {report_csv}")

print("\n" + "=" * 60)
print("🎉 모든 작업 완료!")
print("=" * 60)
print(f"\n📁 생성된 파일:")
print(f"   1. {output_jsonl} ({len(passed_scenarios)}개 통과 항목)")
print(f"   2. {output_json}")
print(f"   3. {report_csv}")
print(f"\n📊 최종 통계:")
print(f"   - Single-turn 통과: {single_turn_passed}/{len(single_turn_results)}개")
print(f"   - Multi-turn 통과: {multi_turn_passed}/{len(multi_turn_results)}개")
print(f"   - 전체 통과율: {passed/total*100:.1f}%" if total > 0 else "   - 전체 통과율: 0%")
print("\n✨정성평가를 진행보세요!")



📊 전체 평가 결과 통합
전체 시나리오: 4개
  - Single Turn: 2개 (통과: 2개)
  - Multi Turn: 2개 (통과: 2개)

✅ 총 통과: 4개 (100.0%)
❌ 총 실패: 0개 (0.0%)

💾 최종 데이터셋 저장
✅ JSONL 저장: outputs/final_dataset.jsonl (통과 항목: 4개)
✅ JSON 저장: outputs/final_dataset.json
✅ CSV 리포트: artifacts/bfcl_evaluation_report.csv

🎉 모든 작업 완료!

📁 생성된 파일:
   1. outputs/final_dataset.jsonl (4개 통과 항목)
   2. outputs/final_dataset.json
   3. artifacts/bfcl_evaluation_report.csv

📊 최종 통계:
   - Single-turn 통과: 2/2개
   - Multi-turn 통과: 2/2개
   - 전체 통과율: 100.0%

✨정성평가를 진행보세요!
